In [98]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from itertools import combinations
from sklearn.preprocessing import StandardScaler

#### Data Analysis

In [99]:
from datetime import datetime
health_df = pd.read_csv('healthcare_dataset.csv')
health_df.head()

,Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
0,Bobby JacksOn,30,Male,B-,Cancer,2024-01-31,Matthew Smith,Sons and Miller,Blue Cross,18856.281306,328,Urgent,2024-02-02,Paracetamol,Normal
1,LesLie TErRy,62,Male,A+,Obesity,2019-08-20,Samantha Davies,Kim Inc,Medicare,33643.327287,265,Emergency,2019-08-26,Ibuprofen,Inconclusive
2,DaNnY sMitH,76,Female,A-,Obesity,2022-09-22,Tiffany Mitchell,Cook PLC,Aetna,27955.096079,205,Emergency,2022-10-07,Aspirin,Normal
3,andrEw waTtS,28,Female,O+,Diabetes,2020-11-18,Kevin Wells,"Hernandez Rogers and Vang,",Medicare,37909.782410,450,Elective,2020-12-18,Ibuprofen,Abnormal
4,adrIENNE bEll,43,Female,AB+,Cancer,2022-09-19,Kathleen Hanna,White-White,Aetna,14238.317814,458,Urgent,2022-10-09,Penicillin,Abnormal


In [100]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from scipy.stats import chi2_contingency
from itertools import product
import scipy.stats as ss

# Dropping unnecessary columns for prediction (e.g., Name, Doctor, Date of Admission, etc.)
health_df.drop_duplicates()
health_df['Discharge Date'] = pd.to_datetime(health_df['Discharge Date'])
health_df['Date of Admission'] = pd.to_datetime(health_df['Date of Admission'])
health_df['Inpatient Days']= (health_df['Discharge Date'] - health_df['Date of Admission']).dt.days+1
features = health_df.drop(columns=['Name', 'Doctor', 'Date of Admission', 'Discharge Date'])

label_encoded_columns = ['Hospital', 'Insurance Provider']
label_encoder = LabelEncoder()

for col in label_encoded_columns:
    features[col] = label_encoder.fit_transform(features[col])


ohe_columns = ['Gender','Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']
ohe = OneHotEncoder()

array_hot_encoded = ohe.fit_transform(features[ohe_columns])
encoded_df = pd.DataFrame(array_hot_encoded.toarray(), columns=ohe.get_feature_names_out(['Gender','Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']))
df = features.join(encoded_df)

df = df[(df['Billing Amount'] > 0) & (df['Inpatient Days'] > 0)]
df["Log Per Day Amount"] = np.log(df["Billing Amount"]/df["Inpatient Days"])
df.drop(['Billing Amount','Gender','Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results'], axis=1, inplace=True)
scaler = StandardScaler()

#df['Age'] = scaler.fit_transform(df[['Age']])
#df['Inpatient Days'] = scaler.fit_transform(df[['Inpatient Days']])

Log Per Day Amount:

In [101]:
from sklearn.feature_selection import mutual_info_regression
indep = ['Age', 'Inpatient Days', 'Hospital', 'Medical Condition_Arthritis', 'Medical Condition_Asthma',
       'Medical Condition_Cancer', 'Medical Condition_Diabetes',
       'Medical Condition_Hypertension', 'Medical Condition_Obesity', 'Room Number', 'Gender_Male','Gender_Female',
       'Blood Type_A+', 'Blood Type_A-', 'Blood Type_AB+', 'Blood Type_AB-',
       'Blood Type_B+', 'Blood Type_B-', 'Blood Type_O+', 'Blood Type_O-',
       'Admission Type_Elective', 'Admission Type_Emergency',
       'Admission Type_Urgent', 'Medication_Aspirin', 'Medication_Ibuprofen',
       'Medication_Lipitor', 'Medication_Paracetamol', 'Medication_Penicillin',
       'Test Results_Abnormal', 'Test Results_Inconclusive',
       'Test Results_Normal']

mutual_info_matrix = np.zeros((len(indep)))
#df = df.drop['Billing Amount']
#df = df.drop['Log Per Day Amount']

# Calculate mutual information for each independent-dependent variable pair
for i, var1 in enumerate(indep):
    X = df[[var1]]
    y = df['Log Per Day Amount']
    mutual_info = mutual_info_regression(X, y)
    mutual_info_matrix[i] = mutual_info[0]  # Store the mutual info scalar

# Convert the matrix to a DataFrame for easier viewing
mutual_info_df = pd.DataFrame(mutual_info_matrix, index=indep, columns=['Log Per Day Amount'])

# Display the mutual information DataFrame
print(mutual_info_df)

                                Log Per Day Amount
Age                                       0.006874
Inpatient Days                            0.644073
Hospital                                  0.100389
Medical Condition_Arthritis               0.020283
Medical Condition_Asthma                  0.023231
Medical Condition_Cancer                  0.021589
Medical Condition_Diabetes                0.019838
Medical Condition_Hypertension            0.019426
Medical Condition_Obesity                 0.018568
Room Number                               0.089002
Gender_Male                               0.036687
Gender_Female                             0.036694
Blood Type_A+                             0.016599
Blood Type_A-                             0.015285
Blood Type_AB+                            0.014758
Blood Type_AB-                            0.014403
Blood Type_B+                             0.015566
Blood Type_B-                             0.018255
Blood Type_O+                  

In [102]:
top_pairs = mutual_info_df.unstack().sort_values(ascending=False).head(32)
print(top_pairs)

Log Per Day Amount  Inpatient Days                    0.644073
                    Hospital                          0.100389
                    Room Number                       0.089002
                    Gender_Female                     0.036694
                    Gender_Male                       0.036687
                    Admission Type_Urgent             0.034001
                    Admission Type_Emergency          0.032511
                    Test Results_Normal               0.031399
                    Test Results_Abnormal             0.029911
                    Admission Type_Elective           0.029761
                    Test Results_Inconclusive         0.027270
                    Medication_Ibuprofen              0.023896
                    Medical Condition_Asthma          0.023231
                    Medication_Paracetamol            0.022952
                    Medical Condition_Cancer          0.021589
                    Medical Condition_Arthritis       0

In [103]:
y = df['Log Per Day Amount']
a =df.drop(columns = ['Log Per Day Amount'])

Linear Regression:

In [104]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error, mean_squared_error

clf = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))


0.7645515031248985
0.8743863580391099
0.3754728017036145
0.8801653022389057
0.7734207804002597
0.8794434492338093
0.37412815226344465
0.8793936060579426


In [105]:
from sklearn.inspection import permutation_importance

result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Test Results_Abnormal: 36966902299911.1094
Test Results_Normal: 36584969631149.8828
Test Results_Inconclusive: 36465615681157.2266
Medication_Aspirin: 15043014101743.2539
Medication_Penicillin: 14926440643080.6367
Medication_Ibuprofen: 14850961404643.9102
Medication_Paracetamol: 14832510861384.5117
Medication_Lipitor: 14645490160152.4434
Gender_Female: 3253054479416.5952
Gender_Male: 3253054451461.8115
Admission Type_Urgent: 1837288508784.6902
Admission Type_Elective: 1820480434907.1746
Admission Type_Emergency: 1816166853298.7688
Blood Type_O+: 1145016696828.1011
Blood Type_B+: 1124517663413.8320
Blood Type_A+: 1124060104991.3672
Blood Type_AB-: 1113536059398.7495
Blood Type_A-: 1105482857445.5334
Blood Type_AB+: 1102462935342.9414
Blood Type_O-: 1085258376803.0433
Blood Type_B-: 1075466432818.4597
Medical Condition_Diabetes: 748948713553.4092
Medical Condition_Cancer: 733140870831.8206
Medical Condition_Arthritis: 730957208629.3397
Medical Condition_Hypertension: 728251375515.9323
Me

In [106]:
b = a[['Test Results_Abnormal', 'Test Results_Normal', 'Test Results_Inconclusive', 'Admission Type_Urgent', 'Admission Type_Elective','Admission Type_Emergency', 'Medication_Aspirin','Medication_Lipitor','Medication_Paracetamol','Medication_Ibuprofen','Medication_Penicillin','Blood Type_O+'
,'Blood Type_O-','Blood Type_B-','Blood Type_AB+','Blood Type_A-','Blood Type_A+','Blood Type_AB-','Blood Type_B+','Medical Condition_Diabetes','Medical Condition_Arthritis','Medical Condition_Obesity','Medical Condition_Hypertension','Medical Condition_Cancer','Medical Condition_Asthma','Inpatient Days']]
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7678689862181041
0.8762813396496036
0.37353673365824835
0.879890460781652
0.7603402705104564
0.8719749253908947
0.3816960023135706
0.8804781108677018


In [107]:
from sklearn.linear_model import Lasso

clf = Lasso(alpha=0.51)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2, random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7718900235464734
0.8785727195551165
0.3702561842103288
0.8795763874673266
0.7645982841133862
0.8744131083837812
0.37823341202997673
0.8801439083248599


In [108]:
result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Inpatient Days: 0.6892
Age: 0.0000
Insurance Provider: 0.0000
Room Number: 0.0000
Gender_Female: 0.0000
Gender_Male: 0.0000
Blood Type_A+: 0.0000
Blood Type_A-: 0.0000
Blood Type_AB+: 0.0000
Blood Type_AB-: 0.0000
Blood Type_B+: 0.0000
Blood Type_B-: 0.0000
Blood Type_O+: 0.0000
Blood Type_O-: 0.0000
Medical Condition_Arthritis: 0.0000
Medical Condition_Asthma: 0.0000
Medical Condition_Cancer: 0.0000
Medical Condition_Diabetes: 0.0000
Medical Condition_Hypertension: 0.0000
Medical Condition_Obesity: 0.0000
Admission Type_Elective: 0.0000
Admission Type_Emergency: 0.0000
Admission Type_Urgent: 0.0000
Medication_Aspirin: 0.0000
Medication_Ibuprofen: 0.0000
Medication_Lipitor: 0.0000
Medication_Paracetamol: 0.0000
Medication_Penicillin: 0.0000
Test Results_Abnormal: 0.0000
Test Results_Inconclusive: 0.0000
Test Results_Normal: 0.0000
Hospital: -0.0001


In [109]:
clf = Lasso(alpha=0.51)
b = a[['Inpatient Days']]
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=52)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7761010311905021
0.880965964830936
0.37040077463680265
0.8792147939169097
0.7468888031861073
0.8642272867632145
0.37850518830271207
0.8816715772264665


Random Forest:

In [110]:
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.13248926446454506
0.3639907477732711
0.8919090901424946
0.9501087618592092
0.727857723659944
0.853145781012802
0.4081106081836834
0.8830590278657514


In [111]:
result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Inpatient Days: 0.9122
Hospital: 0.0431
Room Number: 0.0418
Age: 0.0225
Insurance Provider: 0.0129
Blood Type_O+: 0.0054
Medical Condition_Arthritis: 0.0042
Blood Type_AB+: 0.0040
Medication_Paracetamol: 0.0040
Admission Type_Elective: 0.0039
Test Results_Inconclusive: 0.0037
Medication_Lipitor: 0.0037
Blood Type_A+: 0.0036
Blood Type_O-: 0.0034
Test Results_Normal: 0.0033
Blood Type_B+: 0.0031
Medication_Aspirin: 0.0029
Admission Type_Emergency: 0.0029
Blood Type_B-: 0.0028
Medication_Ibuprofen: 0.0028
Medication_Penicillin: 0.0028
Blood Type_A-: 0.0028
Medical Condition_Cancer: 0.0026
Medical Condition_Obesity: 0.0026
Medical Condition_Diabetes: 0.0026
Medical Condition_Hypertension: 0.0025
Test Results_Abnormal: 0.0025
Gender_Male: 0.0024
Gender_Female: 0.0023
Admission Type_Urgent: 0.0022
Medical Condition_Asthma: 0.0018
Blood Type_AB-: 0.0017


In [112]:
b = a[['Inpatient Days']]
clf = RandomForestRegressor(n_estimators=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7141999553275666
0.8451035175217096
0.41732242756766824
0.8841639214616551
0.7095883641145542
0.8423706809442943
0.4229671381876796
0.884535974367625


KNN:

In [113]:
from sklearn.neighbors import KNeighborsRegressor
clf = KNeighborsRegressor(n_neighbors=49)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

1.1747695464676648
1.0838678639334525
0.04156831375148973
0.8514371311813191
1.2278187985565046
1.1080698527423731
0.0015453593265973842
0.8481165017090996


In [114]:
result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Inpatient Days: 0.0193
Hospital: 0.0054
Room Number: 0.0035
Insurance Provider: 0.0001
Medical Condition_Arthritis: 0.0000
Medication_Aspirin: 0.0000
Admission Type_Urgent: 0.0000
Test Results_Normal: 0.0000
Medication_Penicillin: 0.0000
Medication_Lipitor: 0.0000
Blood Type_O-: 0.0000
Admission Type_Emergency: 0.0000
Medical Condition_Asthma: 0.0000
Medical Condition_Cancer: 0.0000
Blood Type_B+: 0.0000
Blood Type_AB-: -0.0000
Blood Type_A-: -0.0000
Blood Type_A+: -0.0000
Medication_Ibuprofen: -0.0000
Medication_Paracetamol: -0.0000
Admission Type_Elective: -0.0000
Blood Type_B-: -0.0000
Medical Condition_Diabetes: -0.0000
Blood Type_O+: -0.0000
Medical Condition_Obesity: -0.0000
Test Results_Inconclusive: -0.0000
Blood Type_AB+: -0.0000
Test Results_Abnormal: -0.0000
Medical Condition_Hypertension: -0.0000
Gender_Female: -0.0000
Gender_Male: -0.0000
Age: -0.0009


In [115]:
b = a[['Inpatient Days']]
clf = KNeighborsRegressor(n_neighbors=1000)
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7143822740493213
0.8452113783245712
0.41717368346679207
0.8841491372699289
0.7102613541032172
0.8427700481763796
0.4224198668980582
0.8844812329700843


Gradient Boosting:

In [116]:
from sklearn import ensemble
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

clf = ensemble.GradientBoostingRegressor(**params)
X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7047809759240349
0.8395123441165323
0.42500686945638666
0.8849302886442021
0.7107963051114434
0.8430873650526637
0.4219848480521794
0.8844377382417352


In [117]:
result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Inpatient Days: 0.8296
Hospital: 0.0004
Room Number: 0.0003
Insurance Provider: 0.0001
Medical Condition_Arthritis: 0.0001
Medical Condition_Obesity: 0.0001
Blood Type_B+: 0.0000
Blood Type_O+: 0.0000
Medication_Paracetamol: 0.0000
Medical Condition_Cancer: 0.0000
Test Results_Inconclusive: 0.0000
Medication_Ibuprofen: 0.0000
Blood Type_AB+: 0.0000
Test Results_Normal: 0.0000
Blood Type_O-: 0.0000
Admission Type_Elective: 0.0000
Blood Type_B-: 0.0000
Test Results_Abnormal: 0.0000
Medical Condition_Diabetes: 0.0000
Admission Type_Emergency: -0.0000
Medical Condition_Asthma: -0.0000
Gender_Male: -0.0000
Gender_Female: -0.0000
Medication_Aspirin: -0.0000
Blood Type_A+: -0.0000
Medication_Lipitor: -0.0000
Medical Condition_Hypertension: -0.0000
Blood Type_A-: -0.0000
Medication_Penicillin: -0.0000
Admission Type_Urgent: -0.0000
Blood Type_AB-: -0.0000
Age: -0.0002


In [118]:
params = {
    "n_estimators": 500,
    "max_depth": 4,
    "min_samples_split": 5,
    "learning_rate": 0.01,
    "loss": "squared_error",
}

b = a[['Inpatient Days','Hospital','Room Number','Insurance Provider']]

clf = ensemble.GradientBoostingRegressor(**params)
X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2,random_state=42)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7062098967511957
0.8403629553658323
0.4238410893235859
0.8848136976356017
0.7102184089721685
0.8427445692332692
0.42245478961258287
0.8844847253771765


In [119]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(a, y, test_size=0.2, random_state=42)
#X_train = scaler.fit_transform(X_train)
#X_test = scaler.fit_transform(X_test)

#activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
#solver{‘lbfgs’, ‘sgd’, ‘adam’}
clf = MLPRegressor(hidden_layer_sizes=(40,), max_iter=1000, alpha = 0.005, learning_rate_init= 0.002, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)

feature_importance = np.abs(clf.coefs_[0]).sum(axis=1)
sorted_indices = np.argsort(feature_importance)[::-1]
for idx in sorted_indices[:15]:
    print(f"Feature {a.columns[idx]} with importance: {feature_importance[idx]:.4f}")
ypred = clf.predict(X_test)
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))


Feature Test Results_Inconclusive with importance: 10.1288
Feature Test Results_Abnormal with importance: 9.9243
Feature Medication_Paracetamol with importance: 9.6156
Feature Medication_Penicillin with importance: 9.4308
Feature Gender_Male with importance: 9.3852
Feature Admission Type_Emergency with importance: 9.3638
Feature Medical Condition_Hypertension with importance: 9.2559
Feature Medication_Aspirin with importance: 9.2116
Feature Gender_Female with importance: 9.1645
Feature Medical Condition_Arthritis with importance: 9.0996
Feature Admission Type_Elective with importance: 9.0520
Feature Medication_Lipitor with importance: 9.0305
Feature Admission Type_Urgent with importance: 8.9962
Feature Medical Condition_Diabetes with importance: 8.9824
Feature Test Results_Normal with importance: 8.7706
-12.597714796108766
0.4394951361741616


In [ ]:
from sklearn.metrics import mean_squared_error
c = a[['Inpatient Days', 'Admission Type_Urgent', 'Gender_Male','Admission Type_Emergency','Gender_Female','Admission Type_Elective','Test Results_Abnormal','Test Results_Normal',
       ]]

X_train, X_test, y_train, y_test = train_test_split(b, y, test_size=0.2, random_state=42)

X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

#activation{‘identity’, ‘logistic’, ‘tanh’, ‘relu’}
#solver{‘lbfgs’, ‘sgd’, ‘adam’}

clf = MLPRegressor(hidden_layer_sizes=(400,20), max_iter=1000, alpha = 0.002, learning_rate_init= 0.00225, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(1 - (root_mean_squared_error(y_train, ytrainpred) / y_train.mean()))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(1 - (root_mean_squared_error(y_test, ypred) / y_test.mean()))

0.7514725862601008
0.8668751849373131
0.38691368006225246
0.8811797372470866
0.752185107565312
0.8672860586711353
0.38832773029949175
0.8811208153674716


MLP Classifier

In [121]:
bins = [-1, 1, 3, 5, 7, 9, 11]
labels = [0, 2, 4, 6, 8, 10]

df['cat bill'] = pd.cut(df['Log Per Day Amount'], bins=bins, labels=labels, right=True)

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=5, random_state=42)
df['l'] = kmeans.fit_predict(df[['Log Per Day Amount']])

centroids = kmeans.cluster_centers_

cluster_edges = {}
for cluster_label in df['l'].unique():
    cluster_data = df[df['l'] == cluster_label]['Log Per Day Amount']
    min_val = cluster_data.min()
    max_val = cluster_data.max()
    cluster_edges[cluster_label] = (min_val, max_val)

for cluster_label, (min_val, max_val) in cluster_edges.items():
    print(f"Cluster {cluster_label}: Min = {min_val}, Max = {max_val}")
print(df['cat bill'])

Cluster 4: Min = 8.595508199417546, Max = 10.169917635856136
Cluster 1: Min = 7.66657962083134, Max = 8.595031462261742
Cluster 2: Min = 6.818113558841316, Max = 7.666413666294099
Cluster 0: Min = 5.687089475433073, Max = 6.818033255724976
Cluster 3: Min = -0.09144371947623782, Max = 5.686645591100923
0        8
1        8
2        8
3        8
4        6
        ..
55495    4
55496    8
55497    6
55498    8
55499    4
Name: cat bill, Length: 55392, dtype: category
Categories (6, int64): [0 < 2 < 4 < 6 < 8 < 10]


In [ ]:
from sklearn.neural_network import MLPClassifier
X_train, X_test, y_train, y_test = train_test_split(a, df['cat bill'], test_size=0.2, random_state=42)
clf = MLPClassifier(hidden_layer_sizes=(400,20), max_iter=1000, alpha = 0.002, learning_rate_init= 0.00225, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(accuracy_score(y_train, ytrainpred))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(print(accuracy_score(y_test, ypred)))

2.0235145442646627
1.4225029153800222
-0.2581691213265289
0.5909326834111885
2.0196768661431537
1.4211533577144846
-0.2477571610728453
0.5933748533261125
None


In [123]:
result = permutation_importance(clf, X_test, y_test, n_repeats=10, random_state=42)
importance_scores = result.importances_mean
feature_importances = list(zip(X_test.columns, importance_scores))
feature_importances.sort(key=lambda x: x[1], reverse=True)

for feature, importance in feature_importances:
    print(f"{feature}: {importance:.4f}")

Age: 0.0000
Hospital: 0.0000
Insurance Provider: 0.0000
Room Number: 0.0000
Inpatient Days: 0.0000
Gender_Female: 0.0000
Gender_Male: 0.0000
Blood Type_A+: 0.0000
Blood Type_A-: 0.0000
Blood Type_AB+: 0.0000
Blood Type_AB-: 0.0000
Blood Type_B+: 0.0000
Blood Type_B-: 0.0000
Blood Type_O+: 0.0000
Blood Type_O-: 0.0000
Medical Condition_Arthritis: 0.0000
Medical Condition_Asthma: 0.0000
Medical Condition_Cancer: 0.0000
Medical Condition_Diabetes: 0.0000
Medical Condition_Hypertension: 0.0000
Medical Condition_Obesity: 0.0000
Admission Type_Elective: 0.0000
Admission Type_Emergency: 0.0000
Admission Type_Urgent: 0.0000
Medication_Aspirin: 0.0000
Medication_Ibuprofen: 0.0000
Medication_Lipitor: 0.0000
Medication_Paracetamol: 0.0000
Medication_Penicillin: 0.0000
Test Results_Abnormal: 0.0000
Test Results_Inconclusive: 0.0000
Test Results_Normal: 0.0000


In [127]:
from sklearn.neural_network import MLPClassifier
b = a[['Inpatient Days', 'Admission Type_Urgent', 'Gender_Male','Admission Type_Emergency','Gender_Female','Admission Type_Elective','Test Results_Abnormal','Test Results_Normal',
       ]]
X_train, X_test, y_train, y_test = train_test_split(b, df['cat bill'], test_size=0.2, random_state=42)
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)
clf = MLPClassifier(hidden_layer_sizes=(400,20), max_iter=1000, alpha = 0.002, learning_rate_init= 0.00225, activation='relu', solver='adam', random_state=2)
clf.fit(X_train, y_train)
ytrainpred = clf.predict(X_train)
print(mean_squared_error(y_train, ytrainpred))
print(root_mean_squared_error(y_train, ytrainpred))
print(r2_score(y_train, ytrainpred))
print(accuracy_score(y_train, ytrainpred))

ypred = clf.predict(X_test)
print(mean_squared_error(y_test, ypred))
print(root_mean_squared_error(y_test, ypred))
print(r2_score(y_test, ypred))
print(print(accuracy_score(y_test, ypred)))

1.6430392886963194
1.2818109410893321
-0.021599426613083006
0.6393157764087288
1.6311941510966694
1.2771821135204915
-0.007752387151790385
0.6412131058759816
None
